In [2]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from tqdm import tqdm
import os
import pickle
import re
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Importing pickle file

In [3]:
##cambiar a artist
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [4]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [5]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.reset_index(inplace=True)
tracks.head()

,index,url,artist,track
0,0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# Beautiful soup + selenium

To get the info, this script combines selenium to access and search for the different songs and beautifulSoup to get the text info from the tracks.

In [ ]:
almacen_paginas = {
    'pagina' : [], 
    'lyrics':[],
    'featuring' : [],
    'produced_by': [],
    'label': [],
    'written by': [],
    'drums': [],
    'guitar': [],
    'recorded_at' : [],
    'release_date':[],
    'url':[]
}

In [6]:
with open('../data/pickle/genius_lyrics.pickle', 'rb') as almacen_paginas:
    almacen_paginas = pickle.load(almacen_paginas)

In [7]:
#test = tracks.iloc[(864):,:].reset_index(drop=True)
test = tracks.iloc[(len(almacen_paginas['url'])):,:].reset_index(drop=True)
filename = 'genius_lyrics'
test.head()

,index,url,artist,track
0,240831,https://open.spotify.com/track/3ax8cEat0njZGZH...,high n wild,this time
1,240832,https://open.spotify.com/track/1gng5xQBiYCVDkH...,street rappers,estamos bien
2,240889,https://open.spotify.com/track/6E1uX1WP8VtxXfH...,natalie,blue jungle
3,240903,https://open.spotify.com/track/6Yp1rojKvcWzLKV...,cigarettes after sex,crush
4,241090,https://open.spotify.com/track/6pwWIKw85UcUz83...,dr stiig,spooky scary skeletons (remix)


In [8]:
#almacen_paginas['pagina'].append(np.nan)
#almacen_paginas['lyrics'].append(np.nan)
#almacen_paginas['url'].pop()
print(len(almacen_paginas['lyrics']), len(almacen_paginas['pagina']), len(almacen_paginas['url']))

7928 7928 7928


In [49]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
for index, row in test.iterrows():
    print(index, row['artist'].split(',')[0],'---', row['track'].split('-')[0])
    almacen_paginas['url'].append(row['url'])
    if index == 0: 
        driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
        #sleep(1)
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
        #sleep(1)
    else:
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#sticky-nav > div.StickyNavdesktop__Right-sc-60o25r-2.kMDkxm > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
       # sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    try:
        driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
        driver.implicitly_wait(30)
        ## clico el all credits
        try:
            driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[4]/div/a').click()
        except:
            try:
                driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div/a').click()
            except:
                try:
                    driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[2]/div/a').click()
                except:
                    try:
                        driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[1]/div/a').click()
                    except:
                        print('oye que no va')
                        break
        #'//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div/div/a'
        #sleep(1)
        page_source = driver.page_source
        url = driver.current_url
        res = requests.get(url + '#song-info')
        soup = BeautifulSoup(res.content, 'html.parser')
        pagina = soup.text
        almacen_paginas['pagina'].append(pagina)
        try:
            almacen_paginas['lyrics'].append(re.findall(r'Lyrics(.*)EmbedCancel?', pagina)[0])
        except:
            almacen_paginas['lyrics'].append(np.nan)
        with open(f'../data/pickle/{filename}.pickle', 'wb') as data_scrapeado:
            pickle.dump(almacen_paginas, data_scrapeado)
        clear_output(wait=True)
    except:
        almacen_paginas['pagina'].append(np.nan)
        almacen_paginas['lyrics'].append(np.nan)
        clear_output(wait=True)

1 natalie --- blue jungle


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#sticky-nav > div.StickyNavdesktop__Right-sc-60o25r-2.kMDkxm > form > input"}
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00CA6643]
	(No symbol) [0x00C3BE21]
	(No symbol) [0x00B3DA9D]
	(No symbol) [0x00B71342]
	(No symbol) [0x00B7147B]
	(No symbol) [0x00BA8DC2]
	(No symbol) [0x00B8FDC4]
	(No symbol) [0x00BA6B09]
	(No symbol) [0x00B8FB76]
	(No symbol) [0x00B649C1]
	(No symbol) [0x00B65E5D]
	GetHandleVerifier [0x00F1A142+2497106]
	GetHandleVerifier [0x00F485D3+2686691]
	GetHandleVerifier [0x00F4BB9C+2700460]
	GetHandleVerifier [0x00D53B10+635936]
	(No symbol) [0x00C44A1F]
	(No symbol) [0x00C4A418]
	(No symbol) [0x00C4A505]
	(No symbol) [0x00C5508B]
	BaseThreadInitThunk [0x75947D69+25]
	RtlInitializeExceptionChain [0x779FBB9B+107]
	RtlClearBits [0x779FBB1F+191]


# Export

In [ ]:
with open('../data/pickle/genius_lyrics.pickle', 'wb') as data_genius:
    pickle.dump(tracks, data_genius)